In [1]:
import pandas as pd
import numpy as np

# 读取davis_protein_go_vector.csv文件
protein_go_file = 'davis_protein_go_vector.csv'
go_embedding_file = 'GO_IDs_Namespaces_Embedding.csv'

# 加载蛋白质数据（蛋白质ID和GO注释的独热编码）
protein_go_df = pd.read_csv(protein_go_file)

# 加载GO嵌入数据
go_embedding_df = pd.read_csv(go_embedding_file)

# 创建GO_ID -> embedding的映射字典
go_embeddings_dict = dict(zip(go_embedding_df['GO_id'], go_embedding_df['embedding']))

# 1. 将嵌入字符串解析为数值向量
parsed_go_embeddings_dict = {
    go_id: np.fromstring(embedding.strip("[]"), sep=",")
    for go_id, embedding in go_embeddings_dict.items()
}

# 2. 聚合每个蛋白质的GO嵌入
protein_embeddings = []

# 遍历每一行，处理蛋白质 ID 和 GO 注释
for index, row in protein_go_df.iterrows():
    protein_id = row.iloc[0]  # 获取蛋白质 ID
    # 跳过第一列（蛋白质 ID 列），进行布尔索引
    go_ids = protein_go_df.columns[1:][row[1:].astype(bool)].tolist()  # 提取相关联的 GO_ID 列
    embeddings = [
        parsed_go_embeddings_dict[go_id]
        for go_id in go_ids
        if go_id in parsed_go_embeddings_dict
    ]  # 查找对应的嵌入向量
    if embeddings:  # 如果该蛋白质有对应的 GO 嵌入
        summed_embedding = np.sum(embeddings, axis=0)  # 对嵌入向量进行逐元素相加
        avg_embedding = summed_embedding / len(embeddings)  # 除以注释的GO ID总数
    else:
        avg_embedding = np.zeros(len(list(parsed_go_embeddings_dict.values())[0]))  # 如果没有关联 GO，返回零向量
    protein_embeddings.append((protein_id, avg_embedding))

# 3. 构造新的 DataFrame
protein_embedding_df = pd.DataFrame(
    protein_embeddings, columns=["protein_id", "embedding"]
)

# 4. 将嵌入向量展开为单独的列
embedding_columns = [f"embedding_{i+1}" for i in range(len(protein_embeddings[0][1]))]
final_protein_embedding_df = pd.concat(
    [
        protein_embedding_df["protein_id"],
        pd.DataFrame(protein_embedding_df["embedding"].tolist(), columns=embedding_columns),
    ],
    axis=1,
)

# 保存结果为 CSV 文件
final_protein_embedding_df.to_csv("davis_protein_disengo_embeddings_avg.csv", index=False)

print("蛋白质嵌入处理完成，结果已保存为 'davis_protein_disengo_embeddings_avg.csv'")


蛋白质嵌入处理完成，结果已保存为 'davis_protein_disengo_embeddings_avg.csv'
